In [16]:
!pip install opendatasets

In [17]:
import opendatasets as od
od.download ("https://www.kaggle.com/datasets/slothkong/10-monkey-species")

Skipping, found downloaded files in "./10-monkey-species" (use force=True to force download)


In [18]:
DATASET_PATH = "/content/10-monkey-species"

In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt

In [26]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    "/content/10-monkey-species/training/training",
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)


val_data = test_datagen.flow_from_directory(
    "/content/10-monkey-species/validation/validation",
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)



Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [27]:
print(train_data.num_classes)
print(train_data.class_indices)


10
{'n0': 0, 'n1': 1, 'n2': 2, 'n3': 3, 'n4': 4, 'n5': 5, 'n6': 6, 'n7': 7, 'n8': 8, 'n9': 9}


In [28]:
# CNN from Scratch
cnn = Sequential([
    Conv2D(32,3,activation='relu',input_shape=(224,224,3)),
    MaxPooling2D(),

    Conv2D(64,3,activation='relu'),
    MaxPooling2D(),

    Conv2D(128,3,activation='relu'),
    MaxPooling2D(),

    Flatten(),
    Dense(256,activation='relu'),
    Dropout(0.5),
    Dense(10,activation='softmax')
])

In [29]:
cnn.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [30]:
cnn.fit(train_data, validation_data=val_data, epochs=15)


Epoch 1/15
35/35 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.1379 - loss: 2.7988 - val_accuracy: 0.2941 - val_loss: 1.9580
Epoch 2/15
35/35 ━━━━━━━━━━━━━━━━━━━━ 33s 943ms/step - accuracy: 0.2947 - loss: 1.9319 - val_accuracy: 0.4338 - val_loss: 1.5940
Epoch 3/15
35/35 ━━━━━━━━━━━━━━━━━━━━ 29s 837ms/step - accuracy: 0.4315 - loss: 1.5547 - val_accuracy: 0.4890 - val_loss: 1.4379
Epoch 4/15
35/35 ━━━━━━━━━━━━━━━━━━━━ 29s 828ms/step - accuracy: 0.4384 - loss: 1.5326 - val_accuracy: 0.5110 - val_loss: 1.3083
Epoch 5/15
35/35 ━━━━━━━━━━━━━━━━━━━━ 29s 833ms/step - accuracy: 0.5121 - loss: 1.3309 - val_accuracy: 0.5625 - val_loss: 1.2900
Epoch 6/15
35/35 ━━━━━━━━━━━━━━━━━━━━ 29s 818ms/step - accuracy: 0.5432 - loss: 1.2849 - val_accuracy: 0.5993 - val_loss: 1.1509
Epoch 7/15
35/35 ━━━━━━━━━━━━━━━━━━━━ 30s 852ms/step - accuracy: 0.5599 - loss: 1.1946 - val_accuracy: 0.5809 - val_loss: 1.1549
Epoch 8/15
35/35 ━━━━━━━━━━━━━━━━━━━━ 30s 863ms/step - accuracy: 0.5716 - loss: 1.2115 - val_accurac

In [39]:
# MODEL 2 — Transfer Learning --> (ResNet50)
from tensorflow.keras.applications.resnet50 import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)


In [40]:
train_data = train_datagen.flow_from_directory(
    "/content/10-monkey-species/training/training",
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical"
)

val_data = test_datagen.flow_from_directory(
    "/content/10-monkey-species/validation/validation",
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)


Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [41]:
base_model = tf.keras.applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(10, activation="softmax")(x)

model = tf.keras.Model(base_model.input, output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [42]:
model.fit(train_data, validation_data=val_data, epochs=10)

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.1634 - loss: 2.7727 - val_accuracy: 0.7206 - val_loss: 1.3228
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 30s 875ms/step - accuracy: 0.5357 - loss: 1.3753 - val_accuracy: 0.9118 - val_loss: 0.7407
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 31s 898ms/step - accuracy: 0.7291 - loss: 0.8965 - val_accuracy: 0.9338 - val_loss: 0.4593
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 30s 862ms/step - accuracy: 0.8226 - loss: 0.6038 - val_accuracy: 0.9559 - val_loss: 0.3130
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 31s 876ms/step - accuracy: 0.8564 - loss: 0.4855 - val_accuracy: 0.9669 - val_loss: 0.2416
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 30s 881ms/step - accuracy: 0.9105 - loss: 0.3613 - val_accuracy: 0.9669 - val_loss: 0.1937
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 30s 874ms/step - accuracy: 0.9273 - loss: 0.3079 - val_accuracy: 0.9743 - val_loss: 0.1558
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 30s 865ms/step - accuracy: 0.9199 - loss: 0.2783 - val_accurac

In [43]:
base_model.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_4[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [44]:
# Fine Tuning
base_model.trainable = True

for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)



In [45]:
base_model.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_4[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 19,452,928 (74.21 MB)

 Non-trainable params: 4,134,784 (15.77 MB)

In [46]:
model.fit(train_data, validation_data=val_data, epochs=5)

Epoch 1/5
35/35 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.8937 - loss: 0.3876 - val_accuracy: 0.9669 - val_loss: 0.0927
Epoch 2/5
35/35 ━━━━━━━━━━━━━━━━━━━━ 31s 880ms/step - accuracy: 0.9488 - loss: 0.2442 - val_accuracy: 0.9743 - val_loss: 0.0753
Epoch 3/5
35/35 ━━━━━━━━━━━━━━━━━━━━ 31s 875ms/step - accuracy: 0.9608 - loss: 0.1659 - val_accuracy: 0.9816 - val_loss: 0.0624
Epoch 4/5
35/35 ━━━━━━━━━━━━━━━━━━━━ 31s 881ms/step - accuracy: 0.9688 - loss: 0.1445 - val_accuracy: 0.9853 - val_loss: 0.0538
Epoch 5/5
35/35 ━━━━━━━━━━━━━━━━━━━━ 31s 872ms/step - accuracy: 0.9633 - loss: 0.1213 - val_accuracy: 0.9816 - val_loss: 0.0522


In [47]:
loss, acc = model.evaluate(val_data)
print("Final Validation Accuracy:", acc)

9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 409ms/step - accuracy: 0.9812 - loss: 0.0493
Final Validation Accuracy: 0.9816176295280457
